In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import itertools
from functools import total_ordering
from numpy.random import default_rng

#our classes!!! :)
from ParticleSim import Disk
from ParticleSim import Expt

In [3]:
# TEST CELL FOR showAnimation
seed = 42
rng = default_rng(seed=seed) # instatiate the default random number generator
sqrtNumParts = 5

pPosGrid = list(itertools.product(np.linspace(20, 180,sqrtNumParts), np.linspace(20,180,sqrtNumParts)))
pVelGrid = list((rng.random(size=(sqrtNumParts**2,2))-.5)*100) #randomize initial velocities

particles = [Disk(pPosGrid[i], pVelGrid[i]) for i in range(sqrtNumParts**2)]

e1 = Expt(particles, dt=0.001, tmax=0.1, animSpeed=50, updateGraphsEvery=35, potentialType="Coul")

print("Initial KE:",e1.avgKE)
print("Initial PE:",e1.avgPE)
print("Initial Total E:",e1.avgE)
e1.showAnimation1("seed = " + str(seed) + "     ")
print("Final KE:",e1.avgKE)
print("Final PE:",e1.avgPE)
print("Final Total E:",e1.avgE)

Initial KE: 2353.0634724505835
Initial PE: 3984.897222667378
Initial Total E: 6337.960695117961
initializing experiment... 


MovieWriter ffmpeg unavailable; using Pillow instead.


starting simulation... 
[██████████████████████▄▁▁▁▁▁▁▁▁▁]  
finished animating!
Final KE: 2343.000166807649
Final PE: 3995.0637948712597
Final Total E: 6338.0639616789085


In [ ]:
plt.close("all")